# 1. Example of training GNNs for Node Classification

GRB provides easy-to-use APIs to train GNNs, facilitating the entire process from loading graph data, building GNN models, to evaluation and inference. Here is an example for the task of graph classification.

Contents
- [Load Dataset](#Load-Dataset)
- [Build Model](#Build-Model)
- [Training](#Training)
- [Inference](#Inference)
- [Evaluation](#Evaluation)

In [1]:
import os
import torch
import grb.utils as utils

## 1.1. Load Dataset

GRB datasets are named by the prefix *grb-*. There are four *mode* ('easy', 'medium', 'hard', 'full') for test set, representing different average degrees of test nodes, thus different difficulty for attacking them. The node features are processed by *arctan* normalization (first standardization then arctan function), which makes node features fall in the same scale.

In [2]:
from grb.dataset import Dataset

dataset_name = "grb-cora"
dataset = Dataset(name=dataset_name, 
                  data_dir="../data/",
                  mode="full",
                  feat_norm="arctan")

Dataset 'grb-cora' loaded.
    Number of nodes: 2680
    Number of edges: 5148
    Number of features: 302
    Number of classes: 7
    Number of train samples: 1608
    Number of val samples: 268
    Number of test samples: 804
    Dataset mode: full
    Feature range: [-0.9406, 0.9430]


## 1.2. Build Model

GRB supports models based on pure Pytorch, CogDL or DGL. The following is an example of GCN implemented by pure Pytorch. Other models can be found in ``grb/model/torch``, ``grb/model/cogdl``, or ``grb/model/dgl``.

### 1.2.1. GCN (Graph Convolutional Network)

In [3]:
from grb.model.torch import GCN
from grb.utils.normalize import GCNAdjNorm

model_name = "gcn"
model = GCN(in_features=dataset.num_features,
            out_features=dataset.num_classes,
            hidden_features=64, 
            n_layers=3,
            adj_norm_func=GCNAdjNorm,
            layer_norm=True,
            residual=False,
            dropout=0.5)
print("Number of parameters: {}.".format(utils.get_num_params(model)))
print(model)

Number of parameters: 24867.
GCN(
  (layers): ModuleList(
    (0): LayerNorm((302,), eps=1e-05, elementwise_affine=True)
    (1): GCNConv(
      (linear): Linear(in_features=302, out_features=64, bias=True)
      (dropout): Dropout(p=0.5, inplace=False)
    )
    (2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (3): GCNConv(
      (linear): Linear(in_features=64, out_features=64, bias=True)
      (dropout): Dropout(p=0.5, inplace=False)
    )
    (4): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (5): GCNConv(
      (linear): Linear(in_features=64, out_features=7, bias=True)
    )
  )
)


### 1.2.2. GAT (Graph Attention Network)

In [4]:
from grb.model.dgl import GAT

model_name = "gat"
model = GAT(in_features=dataset.num_features,
            out_features=dataset.num_classes,
            hidden_features=64,
            n_layers=3,
            n_heads=4,
            adj_norm_func=None,
            layer_norm=False,
            residual=False,
            feat_dropout=0.6,
            attn_dropout=0.6,
            dropout=0.5)
print("Number of parameters: {}.".format(utils.get_num_params(model)))
print(model)

Number of parameters: 146197.
GAT(
  (layers): ModuleList(
    (0): GATConv(
      (fc): Linear(in_features=302, out_features=256, bias=False)
      (feat_drop): Dropout(p=0.6, inplace=False)
      (attn_drop): Dropout(p=0.6, inplace=False)
      (leaky_relu): LeakyReLU(negative_slope=0.2)
    )
    (1): GATConv(
      (fc): Linear(in_features=256, out_features=256, bias=False)
      (feat_drop): Dropout(p=0.6, inplace=False)
      (attn_drop): Dropout(p=0.6, inplace=False)
      (leaky_relu): LeakyReLU(negative_slope=0.2)
    )
    (2): GATConv(
      (fc): Linear(in_features=256, out_features=7, bias=False)
      (feat_drop): Dropout(p=0.0, inplace=False)
      (attn_drop): Dropout(p=0.0, inplace=False)
      (leaky_relu): LeakyReLU(negative_slope=0.2)
    )
  )
  (dropout): Dropout(p=0.5, inplace=False)
)


Using backend: pytorch


### 1.2.3. APPNP (Approximated Personalized Propagation of Neural Predictions)

In [5]:
from grb.model.torch import APPNP
from grb.utils.normalize import GCNAdjNorm

model_name = "appnp"
model = APPNP(in_features=dataset.num_features,
              out_features=dataset.num_classes,
              hidden_features=128, 
              n_layers=3,
              adj_norm_func=GCNAdjNorm,
              layer_norm=False,
              edge_drop=0.1,
              alpha=0.01,
              k=3,
              dropout=0.5)
print("Number of parameters: {}.".format(utils.get_num_params(model)))
print(model)

Number of parameters: 56199.
APPNP(
  (layers): ModuleList(
    (0): Linear(in_features=302, out_features=128, bias=True)
    (1): Linear(in_features=128, out_features=128, bias=True)
    (2): Linear(in_features=128, out_features=7, bias=True)
  )
  (edge_dropout): SparseEdgeDrop()
  (dropout): Dropout(p=0.5, inplace=False)
)


### 1.2.4. GIN (Graph Isomorph Network)

In [6]:
from grb.model.torch import GIN

model_name = "gin"
model = GIN(in_features=dataset.num_features,
            out_features=dataset.num_classes,
            hidden_features=64, 
            n_layers=3,
            adj_norm_func=None,
            layer_norm=False,
            batch_norm=True,
            dropout=0.5)
print("Number of parameters: {}.".format(utils.get_num_params(model)))
print(model)

Number of parameters: 36745.
GIN(
  (layers): ModuleList(
    (0): GINConv(
      (linear1): Linear(in_features=302, out_features=64, bias=True)
      (linear2): Linear(in_features=64, out_features=64, bias=True)
      (norm): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (dropout): Dropout(p=0.5, inplace=False)
    )
    (1): GINConv(
      (linear1): Linear(in_features=64, out_features=64, bias=True)
      (linear2): Linear(in_features=64, out_features=64, bias=True)
      (norm): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (dropout): Dropout(p=0.5, inplace=False)
    )
  )
  (mlp_layers): ModuleList(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): Linear(in_features=64, out_features=7, bias=True)
  )
  (dropout): Dropout(p=0.5, inplace=False)
)


### 1.2.5. GraphSAGE

In [7]:
from grb.model.torch import GraphSAGE
from grb.utils.normalize import SAGEAdjNorm

model_name = "graphsage"
model = GraphSAGE(in_features=dataset.num_features,
                  out_features=dataset.num_classes,
                  hidden_features=64,
                  n_layers=3,
                  adj_norm_func=SAGEAdjNorm,
                  layer_norm=False,
                  dropout=0.5)
print("Number of parameters: {}.".format(utils.get_num_params(model)))
print(model)

Number of parameters: 147840.
GraphSAGE(
  (layers): ModuleList(
    (0): SAGEConv(
      (pool_layer): Linear(in_features=302, out_features=302, bias=True)
      (linear1): Linear(in_features=302, out_features=64, bias=True)
      (linear2): Linear(in_features=302, out_features=64, bias=True)
      (dropout): Dropout(p=0.5, inplace=False)
    )
    (1): SAGEConv(
      (pool_layer): Linear(in_features=64, out_features=64, bias=True)
      (linear1): Linear(in_features=64, out_features=64, bias=True)
      (linear2): Linear(in_features=64, out_features=64, bias=True)
      (dropout): Dropout(p=0.5, inplace=False)
    )
    (2): SAGEConv(
      (pool_layer): Linear(in_features=64, out_features=64, bias=True)
      (linear1): Linear(in_features=64, out_features=7, bias=True)
      (linear2): Linear(in_features=64, out_features=7, bias=True)
    )
  )
)


### 1.2.6. SGCN (Simplified Graph Convolutional Network)

In [8]:
from grb.model.torch import SGCN
from grb.utils.normalize import GCNAdjNorm

model_name = "sgcn"
model = SGCN(in_features=dataset.num_features,
             out_features=dataset.num_classes,
             hidden_features=64,
             n_layers=3,
             adj_norm_func=GCNAdjNorm,
             k=4,
             dropout=0.5)
print("Number of parameters: {}.".format(utils.get_num_params(model)))
print(model)

Number of parameters: 24611.
SGCN(
  (batch_norm): BatchNorm1d(302, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (in_conv): Linear(in_features=302, out_features=64, bias=True)
  (out_conv): Linear(in_features=64, out_features=7, bias=True)
  (layers): ModuleList(
    (0): SGConv(
      (linear): Linear(in_features=64, out_features=64, bias=True)
    )
  )
  (dropout): Dropout(p=0.5, inplace=False)
)


### 1.2.7. TAGCN (Topological Adaptive Graph Convolutional Network)

In [9]:
from grb.model.torch import TAGCN
from grb.utils.normalize import GCNAdjNorm

model_name = "tagcn"
model = TAGCN(in_features=dataset.num_features,
              out_features=dataset.num_classes,
              hidden_features=64,
              n_layers=3,
              adj_norm_func=GCNAdjNorm,
              k=2,
              dropout=0.5)
print("Number of parameters: {}.".format(utils.get_num_params(model)))
print(model)

Number of parameters: 71751.
TAGCN(
  (layers): ModuleList(
    (0): TAGConv(
      (linear): Linear(in_features=906, out_features=64, bias=True)
      (dropout): Dropout(p=0.5, inplace=False)
    )
    (1): TAGConv(
      (linear): Linear(in_features=192, out_features=64, bias=True)
      (dropout): Dropout(p=0.5, inplace=False)
    )
    (2): TAGConv(
      (linear): Linear(in_features=192, out_features=7, bias=True)
    )
  )
)


### 1.2.8. MLP (Multi-layer Perceptron)

In [10]:
from grb.model.torch import MLP

model_name = "mlp"
model = MLP(in_features=dataset.num_features,
            out_features=dataset.num_classes,
            hidden_features=64, 
            n_layers=3,
            dropout=0.5)
print("Number of parameters: {}.".format(utils.get_num_params(model)))
print(model)

Number of parameters: 24007.
MLP(
  (layers): ModuleList(
    (0): MLPLayer(
      (linear): Linear(in_features=302, out_features=64, bias=True)
      (dropout): Dropout(p=0.5, inplace=False)
    )
    (1): MLPLayer(
      (linear): Linear(in_features=64, out_features=64, bias=True)
      (dropout): Dropout(p=0.5, inplace=False)
    )
    (2): MLPLayer(
      (linear): Linear(in_features=64, out_features=7, bias=True)
    )
  )
)


## 1.3. Training

GRB provides ``grb.utils.trainer`` that facilitates the training process of GNNs. The training mode can be chosen from ``inductive`` or ``transductive``. In the inductive mode, only train nodes can be seen during training, train+val nodes can be seen during validation, train+val+test nodes can be seen during testing. In the transductive mode, all nodes are available for each process.  

In [5]:
save_dir = "./saved_models/{}/{}".format(dataset_name, model_name)
save_name = "model.pt"
device = "cuda:0"
feat_norm = None
train_mode = "inductive"  # "transductive"

In [6]:
from grb.trainer.trainer import Trainer

trainer = Trainer(dataset=dataset, 
                  optimizer=torch.optim.Adam(model.parameters(), lr=0.01),
                  loss=torch.nn.functional.cross_entropy,
                  lr_scheduler=False,
                  early_stop=True,
                  early_stop_patience=500,
                  feat_norm=feat_norm,
                  device=device)

In [7]:
trainer.train(model=model, 
              n_epoch=2000,
              eval_every=1,
              save_after=0,
              save_dir=save_dir,
              save_name=save_name,
              train_mode=train_mode,
              verbose=False)

  0%|          | 0/2000 [00:00<?, ?it/s]

Training early stopped. Best validation score: 0.8134
Training runtime: 19.9552.


## 1.4. Inference

In [8]:
model = torch.load(os.path.join(save_dir, save_name))
model = model.to(device)
model.eval()

GAT(
  (layers): ModuleList(
    (0): GATConv(
      (fc): Linear(in_features=302, out_features=256, bias=False)
      (feat_drop): Dropout(p=0.6, inplace=False)
      (attn_drop): Dropout(p=0.6, inplace=False)
      (leaky_relu): LeakyReLU(negative_slope=0.2)
    )
    (1): GATConv(
      (fc): Linear(in_features=256, out_features=256, bias=False)
      (feat_drop): Dropout(p=0.6, inplace=False)
      (attn_drop): Dropout(p=0.6, inplace=False)
      (leaky_relu): LeakyReLU(negative_slope=0.2)
    )
    (2): GATConv(
      (fc): Linear(in_features=256, out_features=7, bias=False)
      (feat_drop): Dropout(p=0.0, inplace=False)
      (attn_drop): Dropout(p=0.0, inplace=False)
      (leaky_relu): LeakyReLU(negative_slope=0.2)
    )
  )
  (dropout): Dropout(p=0.5, inplace=False)
)

In [9]:
# by trainer
pred = trainer.inference(model)

In [10]:
# by utils
pred = utils.inference(model, 
                       features=dataset.features,
                       feat_norm=feat_norm,
                       adj=dataset.adj,
                       adj_norm_func=model.adj_norm_func,
                       device=device)

## 1.5. Evaluation

In [11]:
# by trainer
test_score = trainer.evaluate(model, dataset.test_mask)
print("Test score: {:.4f}".format(test_score))

Test score: 0.8458


In [12]:
# by utils
test_score = utils.evaluate(model, 
                            features=dataset.features,
                            adj=dataset.adj,
                            labels=dataset.labels,
                            feat_norm=feat_norm,
                            adj_norm_func=model.adj_norm_func,
                            mask=dataset.test_mask,
                            device=device)
print("Test score: {:.4f}".format(test_score))

Test score: 0.8458


For further information, please refer to the [GRB Documentation](https://grb.readthedocs.io/en/latest/).